## Assignment 1: Word Count

In this assignment, you will identify the most popular words appearing in the paper Detecting Covid19 Fake News. You will create a word cloud to visualize the results. You will first upload the file covid19_fake_news_detection.txt to the hpc server. Please complete the five questions at this notebook.

In [2]:
from pyspark.sql import SparkSession

# change the account name to your email account
account='sli'

# define a root path to access the data in isa460 folder
root_path='/net/clusterhn/home/'+account+'/isa460/Data/'

spark = SparkSession.builder.appName("My First Spark Program")\
        .config("spark.port.maxRetries", "100")\
        .getOrCreate()

# confiture the log level (defaulty is WWARN)
spark.sparkContext.setLogLevel('ERROR')

# read the csv file

book = spark.read.text(root_path+"covid19_fake_news_detection.txt")

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from helper_functions import displayByGroup
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# create a spark session
spark = SparkSession.builder.appName("Boston Crime Analysis")\
        .config("spark.port.maxRetries", "100")\
        .getOrCreate()

# confiture the log level (defaulty is WWARN)
spark.sparkContext.setLogLevel('ERROR')

# read the csv file, remove year with null value and remove current month (Septmber 2023)
crime = spark.read.csv("/opt/shared/boston_crime", header=True, inferSchema=True).filter(F.isnull(F.col('year'))==False)\
       .filter(F.date_format(F.col('occurred_on_date'), 'yyyy-MM')!='2023-09')

# change all columns to lower case

crime=crime.toDF(*[x.lower() for x in crime.columns])

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/18 06:45:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
book.show(10, truncate=50)

+--------------------------------------------------+
|                                             value|
+--------------------------------------------------+
| Detecting Covid-19 Misinformation on Social Media|
|Jason Michaud, Bryant University, jmichaud1@bry...|
|      Suhong Li, Bryant University, sli@bryant.eud|
|                                                  |
|                                          Abstract|
|                                                  |
|There have been many studies conducted over the...|
|                                                  |
|Keywords: Misinformation, COVID-19, Machine Lea...|
|                                                  |
+--------------------------------------------------+
only showing top 10 rows



### Below code shows the top 10 words based on frequency after changing all words to lowercase, remove null value and special characters)

In [17]:
from pyspark.sql.functions import (
    col,
    explode,
    lower,
    regexp_extract,
    split,
)

lines = book.select(split(book.value, " ").alias("line"))
 
words = lines.select(explode(col("line")).alias("word"))
 
words_lower = words.select(lower(col("word")).alias("word"))
 
words_clean = words_lower.select(
    regexp_extract(col("word"), "[a-z']*", 0).alias("word")
)
 
words_nonull = words_clean.where(col("word") != "")
 
results = words_nonull.groupby(col("word")).count()
 
results.orderBy("count", ascending=False).show(10)

+-------+-----+
|   word|count|
+-------+-----+
|    the|  313|
|    and|  159|
|     of|  131|
|     in|  117|
|     is|  115|
|      a|  114|
|    for|  111|
|     to|  107|
|   news|   99|
|   this|   91|
|   fake|   79|
|   that|   78|
|  covid|   76|
|     as|   68|
|    was|   63|
| united|   61|
|     be|   55|
|   real|   52|
|     it|   47|
|dataset|   46|
+-------+-----+
only showing top 20 rows



## 1. Modify the orignal code to return the number of the distinct words in this article

## 2. Modify the origianl code to return a smaple of 10 words that only appreas once

## 3. Modify original code to remove all words having less than 3 letters, show top 20 words.

## 4. Modify the above code to remove the stopwords (the, and, for, this, that, was, are) after removing words having less than 3 letters,  save the top 100 words in a dataframe

## 5. Create a word cloud to visualize the top 100 words (hint: you may need to install other package, to install a package within the notebook, use 
!pip install packagename